# Lesson 1 Notebook
In this notebook we will work with documents as vectors in a high dimensional space. We will look at some different ways of doing this using well established methods.

In [ ]:
import zipfile

## The data

In this notebook we will be working with comparing documents. We will work with real world data in the form of *patent applications*. These are documents which are submitted to patent organizations (in our case the European Patent Office, EPO) when requesting a patent. 

Published patent applications are open, so is a good source of example data for natural language processing.

## Bag-of-Words